# **Klue-bert-base with tensorflow**

- 구글 드라이브 연동

In [1]:
import os
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


## Klue-bert-base 모델 로드
🤗 **[Huggingface transformers API](https://huggingface.co/beomi/kcbert-base)** 사용
- HuggingFace Model Hub는 코드 공유 저장소인 github와 유사하게 개인 혹은 기관에서 학습한 언어모델을 공유하는 모델 저장소입니다.
- PyTorch, Tensorflow, JAX, ONNX 등 다양한 딥러닝 프레임워크를 지원합니다.



**다 대 일 유형** 
- 진행하는 태스크는 다수의 레이블(8개의 감정) 중 하나의 출력(한 개의 메인 감정)이 필요한 다대일 태스크입니다. 

**TFBertSequenceClassification**
- tf.keras.Model 하위 클래스
- 허깅 페이스에서 제공하는 다대일 유형의 모델 클래스 구현체를 사용합니다.

In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 54.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 115.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 21.8 MB/s eta 0:00:00


In [3]:
from transformers import BertTokenizer, TFBertForSequenceClassification
MODEL_NAME = "beomi/kcbert-base"
model = TFBertForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=8, from_pt=True)
tokenizer = BertTokenizer.from_pretrained(MODEL_NAME)

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
# 전달한 분류 레이블 개수만큼의 출력층이 설계되었습니다. 
model.config

BertConfig {
  "_name_or_path": "beomi/kcbert-base",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5,
    "LABEL_6": 6,
    "LABEL_7": 7
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 300,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",

## TensorFlow 딥러닝 프레임워크

옵티마이저 : RAdam(Rectified Adam)

In [5]:
!pip install tensorflow_addons # tensorflow에서 RAdam optimizer를 사용하기 위해 필요한 패키지 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 17.4 MB/s eta 0:00:00


In [6]:
import os
import pandas as pd
import numpy as np
import re
from tqdm import tqdm
import urllib.request
import tensorflow_addons as tfa 
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split

**gpu 환경에서 학습**



In [7]:
device_name = tf.test.gpu_device_name()
if device_name == '/device:GPU:0':
  print("GPU 작동 중")
  mirrored_strategy = tf.distribute.MirroredStrategy()
else:
  print("GPU 미작동 중")

GPU 작동 중


## 학습/검증 데이터셋

In [8]:
dataset = pd.read_csv('/content/drive/MyDrive/Team_MLS/data/data/6차데이터.csv')

In [9]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
dataset['감정'] = le.fit_transform(np.array(dataset['감정']))
dataset['감정'] = dataset['감정'].astype('int32')

In [10]:
dataset

,new_text,감정
0,맛이 없더라도 맛있다고 한 마디 해주면 덧나나? 이 화를 어떻게 풀어야 할지 고민 ...,0
1,학교에서 하는 상담이 어떤 의미가 있는지 모르겠어. 평소 감정이 있는 어린 부장이 ...,0
2,나 요새 기분이 매우 안 좋아. 내 친구가 내 험담을 하고 다녔더라고. 일도 못하는...,0
3,코로나 일구 때문에 짜증 나는 일이 너무 많아. 아들은 내 걱정보다 보험만 궁금해 ...,0
4,선물로 누나에게 머리띠를 선물했는데 쓰레기통에서 봤어. 내가 버려진 것 같은 기분이...,0
...,...,...
208124,너무 마음에 들지! 내가 비싸서 못 샀던 거야. 지나가다가 이벤트 하는 걸 보고 신...,4
208125,와우! 날 위해서 그렇게까지 해주는 거야? 정말 고마워. 나를 위해서 이렇게 영화랑...,4
208126,다음에 너도 만나자. 좋아. 스트레스 좀 풀고 싶다. 함께 달리는 크루가 있어서 지...,4
208127,함께 달리는 크루가 있어서 내가 지치고 힘들 때 큰 힘이 되어주고 있어. 아니야. ...,4


### Train/Test 분리(레이블 비율 반영)
- 8 : 2 비율로 분리
- train_test_split에 stratify 값을 target 데이터로 지정하여 각각의 클래스 비율을 trainset과 testset에 유지하여 분할하도록 합니다. 


In [11]:
# 입출력 데이터 분리
X_data = dataset['new_text']
y_data = dataset['감정']

In [12]:
TEST_SIZE = 0.2 
RANDOM_STATE = 77

# stratify에 target 데이터를 전달하여 데이터셋의 레이블 별 분포를 고려하여 train, test 데이터 분리하도록 설정하였습니다. 
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, 
                                                    test_size = TEST_SIZE, 
                                                    random_state = RANDOM_STATE, 
                                                    stratify = y_data)

In [13]:
print(f"훈련 입력 데이터 개수: {len(X_train)}")
print(f"테스트 입력 데이터 개수: {len(X_test)}")

훈련 입력 데이터 개수: 166503
테스트 입력 데이터 개수: 41626


### 모델 input type으로 변환
- 토큰화, 타입 변환
- TFBertForSequenceClassification 클래스는 tf.keras.Model 하위 클래스이므로, tensorflow의 keras 사용

- 모델 입력 형식
- model([input_ids, attention_mask, token_type_ids])
- model([input_ids, attention_mask])

In [14]:
# 입력 데이터(문장) 길이 제한
MAX_SEQ_LEN = 64

In [15]:
def convert_data(X_data, y_data):
    # BERT 입력으로 들어가는 token, mask, segment, target 저장용 리스트
    tokens, masks, segments, targets = [], [], [], []
    for X, y in tqdm(zip(X_data, y_data)):
        # token: 입력 문장 토큰화
        # token = tokenizer(X, truncation = True, padding = 'max_length', max_length = MAX_SEQ_LEN)
        token = tokenizer.encode(X, truncation = True, padding = 'max_length', max_length = MAX_SEQ_LEN)

        # Mask: 토큰화한 문장 내 패딩이 아닌 경우 1, 패딩인 경우 0으로 초기화
        num_zeros = token.count(0)
        mask = [1] * (MAX_SEQ_LEN - num_zeros) + [0] * num_zeros
        
        # segment: 문장 전후관계 구분: 오직 한 문장이므로 모두 0으로 초기화
        segment = [0] * MAX_SEQ_LEN

        tokens.append(token)
        masks.append(mask)
        segments.append(segment)
        targets.append(y)

    tokens = np.array(token)
    masks = np.array(masks)
    segments = np.array(segments)
    targets = np.array(targets)


    return [tokens,masks,segments], targets

In [16]:
# train 데이터를 Bert의 Input 타입에 맞게 변환
train_x, train_y = convert_data(X_train, y_train)

166503it [02:58, 934.31it/s]


In [17]:
# test 데이터를 Bert의 Input 타입에 맞게 변환
test_x, test_y = convert_data(X_test, y_test)

41626it [00:44, 938.99it/s]


# BERT를 활용한 파인튜닝

In [19]:
# token, mask, segment 입력 정의
token_inputs = tf.keras.layers.Input((MAX_SEQ_LEN,), dtype = tf.int32, name = 'input_word_ids')
mask_inputs = tf.keras.layers.Input((MAX_SEQ_LEN,), dtype = tf.int32, name = 'input_masks')
segment_inputs = tf.keras.layers.Input((MAX_SEQ_LEN,), dtype = tf.int32, name = 'input_segment')
bert_outputs = model([token_inputs, mask_inputs, segment_inputs])

In [20]:
bert_outputs

TFSequenceClassifierOutput(loss=None, logits=<KerasTensor: shape=(None, 8) dtype=float32 (created by layer 'tf_bert_for_sequence_classification')>, hidden_states=None, attentions=None)

In [21]:
bert_output = bert_outputs[0]

## 감정 분류 모델 컴파일

In [22]:
DROPOUT_RATE = 0.5
NUM_CLASS = 8
dropout = tf.keras.layers.Dropout(DROPOUT_RATE)(bert_output)
# Multi-class classification 문제이므로 activation function은 softmax로 설정
sentiment_layer = tf.keras.layers.Dense(NUM_CLASS, activation='softmax', kernel_initializer = tf.keras.initializers.TruncatedNormal(stddev=0.02))(dropout)
sentiment_model = tf.keras.Model([token_inputs, mask_inputs, segment_inputs], sentiment_layer)

In [23]:
# 옵티마이저 Rectified Adam 하이퍼파리미터 조정
OPTIMIZER_NAME = 'RAdam'
LEARNING_RATE = 5e-5
TOTAL_STEPS = 10000
MIN_LR = 1e-5
WARMUP_PROPORTION = 0.1
EPSILON = 1e-8
CLIPNORM = 1.0
optimizer = tfa.optimizers.RectifiedAdam(learning_rate = LEARNING_RATE,
                                          total_steps = TOTAL_STEPS, 
                                          warmup_proportion = WARMUP_PROPORTION, 
                                          min_lr = MIN_LR, 
                                          epsilon = EPSILON,
                                          clipnorm = CLIPNORM)

In [24]:
# 감정분류 모델 컴파일
sentiment_model.compile(optimizer = optimizer, 
                        loss = tf.keras.losses.SparseCategoricalCrossentropy(), 
                        metrics = ['accuracy'])

## 조기종료 조건

In [25]:
MIN_DELTA = 1e-3
PATIENCE = 5

early_stopping = EarlyStopping(
    monitor = "val_accuracy", 
    min_delta = MIN_DELTA,
    patience = PATIENCE)

## 최고 성능 모델 저장

In [ ]:
# 최고 성능의 모델 파일을 저장할 이름과 경로 설정
BEST_MODEL_NAME = '/content/drive/MyDrive/moa/loss/DATA05_Klue_bert_best_model.h5'

In [ ]:
model_checkpoint = ModelCheckpoint(
    filepath = BEST_MODEL_NAME,
    monitor = "val_loss",
    mode = "min",
    save_best_only = True, # 성능 향상 시에만 모델 저장
    verbose = 1
)

In [ ]:
callbacks = [early_stopping, model_checkpoint]

## 감정 분류 모델 학습

In [ ]:
EPOCHS = 100
BATCH_SZIE = 32

In [ ]:
sentiment_model.fit(train_x, train_y, 
                    epochs = EPOCHS, 
                    shuffle = True, 
                    batch_size = BATCH_SZIE, 
                    validation_data = (test_x, test_y),
                    callbacks = callbacks
                    )